# **BUILDING MODELS**

Having analysed, cleaned and split the dataset, mo

    1. Model 1
        A. Build Model
        B. Predictions of the Model
        C. Feature Importance from the Model
        D. Create Pickle File
    2. Model 2
        A. Build Model
        B. Predictions of the Model
        C. Feature Importance from the Model
        D. Create Pickle File

## **1. Building Model 1**